In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from openai import OpenAI
from langchain_chroma.vectorstores import Chroma
import chromadb
import re
from langchain_community.document_loaders import Docx2txtLoader
from glob import glob
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from huggingface_hub import hf_hub_download
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import BaseChatMessageHistory, RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
from langchain import hub
from chromadb.utils import embedding_functions

### 0. 환경 설정

In [2]:
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')

In [ ]:
# !pip install pypdf

In [ ]:
# chroma_client = chromadb.PersistentClient("db")
# col = chroma_client.get_or_create_collection(
#     name= "database",
#     embedding_function=ef
# )

In [ ]:
# chroma_client.delete_collection("database")

In [ ]:
# vector_store = Chroma(
#     client=client,
#     collection_name="housing",
#     embedding_function=emb,
# )
# retriever = vector_store.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 5, "filter": {"승강기": True, "월임대료": {"$lte": 400000}}}
# )

In [ ]:
# # MMR + 넓은 후보 집합
# # MMR : 질의와의 유사도는 높이고, 서로 간의 중복은 줄이는 방식
# results = vector_store.max_marginal_relevance_search(
#     "청년전용 보증부월세대출의 대출 대상 연령은?",
#     k=3, fetch_k=50,  # 후보 넓게, 최종 3개
# )
# for doc in results:
#     print(doc.page_content, doc.metadata)

[대환대출]

월세 부담 완화를 위해 대출해 드립니다. 대출대상:

청년전용 보증부월세대출 이용중인 자로서 보증금 대출 상환 후 월세대출에 대한 대환대출이 필요한 무주택 세대주

대출금리: 1%

대출한도: 1,200만원 이내

대출기간: 2년(최장 8년 이용가능)

  ※  자세한 보증 요건 및 기한은 신청 수탁은행에 문의 필요

1. 대출 대상

청년전용 보증부월세대출을 이용중인 자 중 보증금 대출을 상환하고 월세 대출에 대한 상환 유예가 필요한 자

2. 신청시기

임대차계약종료 1개월전부터 청년전용 보증부월세대출 만기일 전까지 신청

3. 대출한도

다음 중 작은 금액으로 산정

1. 대출한도

1,200만원 이내

2. 소요자금에 대한 대출비율

기존 청년전용 보증부월세대출 잔액범위 이내

4. 대출금리

연 1.0%(단, 청년전용 보증부월세대출 이용시 적용된 가산금리는 유지)

5. {'source': './loan_data/1.청년전용_보증부월세대출.docx'}
대출한도

1,200만원 이내

2. 소요자금에 대한 대출비율

기존 청년전용 보증부월세대출 잔액범위 이내

4. 대출금리

연 1.0%(단, 청년전용 보증부월세대출 이용시 적용된 가산금리는 유지)

5. 이용기간

2년(최장 10년 4개월 가능)

6. 담보취득

주택도시보증공사 청년전용 월세대출보증서

7. 중복대출허용

청년전용 보증부월세대출 대환을 이용중인 경우 주택도시기금 전세자금대출 신청이 가능

8. 고객부담비용

인지세 : 고객/은행 각 50% 부담

보증서 담보 취급 시 보증료

9. {'source': './loan_data/1.청년전용_보증부월세대출.docx'}
[출처] 주택도시기금 청년 전월세보증금 대출

[링크] https://nhuf.molit.go.kr/FP/FP05/FP0502/FP05020701.jsp

[상품명] 청년전용 보증부월세대출

주택도시기금의 개인상품 중 청년전용 보증부월세대출입니다. 청년들에게 저리로 전월세보증금 및 월세를 대출해드립니다. [대출대

In [ ]:
# # MMR + 넓은 후보 집합
# # MMR : 질의와의 유사도는 높이고, 서로 간의 중복은 줄이는 방식
# results = vector_store.similarity_search(
#     "청년전용 보증부월세대출의 대출 대상 연령은?",
#     k=3 # 후보 넓게, 최종 3개
# )
# for doc in results:
#     print(doc.page_content, doc.metadata)

[대환대출]

월세 부담 완화를 위해 대출해 드립니다. 대출대상:

청년전용 보증부월세대출 이용중인 자로서 보증금 대출 상환 후 월세대출에 대한 대환대출이 필요한 무주택 세대주

대출금리: 1%

대출한도: 1,200만원 이내

대출기간: 2년(최장 8년 이용가능)

  ※  자세한 보증 요건 및 기한은 신청 수탁은행에 문의 필요

1. 대출 대상

청년전용 보증부월세대출을 이용중인 자 중 보증금 대출을 상환하고 월세 대출에 대한 상환 유예가 필요한 자

2. 신청시기

임대차계약종료 1개월전부터 청년전용 보증부월세대출 만기일 전까지 신청

3. 대출한도

다음 중 작은 금액으로 산정

1. 대출한도

1,200만원 이내

2. 소요자금에 대한 대출비율

기존 청년전용 보증부월세대출 잔액범위 이내

4. 대출금리

연 1.0%(단, 청년전용 보증부월세대출 이용시 적용된 가산금리는 유지)

5. {'source': './loan_data/1.청년전용_보증부월세대출.docx'}
[대환대출]

월세 부담 완화를 위해 대출해 드립니다. 대출대상:

청년전용 보증부월세대출 이용중인 자로서 보증금 대출 상환 후 월세대출에 대한 대환대출이 필요한 무주택 세대주

대출금리: 1%

대출한도: 1,200만원 이내

대출기간: 2년(최장 8년 이용가능)

  ※  자세한 보증 요건 및 기한은 신청 수탁은행에 문의 필요

1. 대출 대상

청년전용 보증부월세대출을 이용중인 자 중 보증금 대출을 상환하고 월세 대출에 대한 상환 유예가 필요한 자

2. 신청시기

임대차계약종료 1개월전부터 청년전용 보증부월세대출 만기일 전까지 신청

3. 대출한도

다음 중 작은 금액으로 산정

1. 대출한도

1,200만원 이내

2. 소요자금에 대한 대출비율

기존 청년전용 보증부월세대출 잔액범위 이내

4. 대출금리

연 1.0%(단, 청년전용 보증부월세대출 이용시 적용된 가산금리는 유지)

5. {'source': './loan_data/1.청년전용_보증부월세대출.docx'}
[대환대출]



In [ ]:
# docs = vector_store.similarity_search(
#     "임대료 정보 알려줘",
#     k=5,
#     filter={"구": "강서구", "주택유형": "아파트"}
# )

##### 3. 모델 로드

In [8]:
repo_id    = "teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf"   # gguf 파일이 올라간 레포
revision   = "main"         # 또는 커밋 해시
filename   = "EEVE-Korean-Instruct-10.8B-v1.0-Q5_K_M.gguf"  # 정확한 파일명

gguf_path = hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    revision=revision,       
    token=HF_TOKEN           
)

In [9]:
llm = ChatOllama(model = 'eeve-q5km:latest')
retriever = vector_store.as_retriever()
prompt = hub.pull('rlm/rag-prompt')     # Langchain Hub에서 RAG 전용 프롬프트로 가장 많이 사용되는 prompt 호출

In [10]:
# Retriever로 검색한 유사 문서의 내용을 하나의 String으로 결합
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

#### 4. RAG 구축

In [11]:
rag_chain = (
{'context' : retriever | format_docs , 'question' : RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("나는 청년이고, 청년 주택을 찾고 있는데, 강남구에 있는 공실 주택 정보를 줘"):
    print(chunk,end='', flush=True)

청
년
으로서
 강남
구에
 있는
 공
실
 주택
 정보를
 찾고
 계신
다면
,
 해당
 지역에서
 청년
주택
을
 찾
는데
 도움을
 줄
 수
 있는
 정부
 지원
 프로
그


KeyboardInterrupt: 

In [ ]:
# prompt = ChatPromptTemplate.from_template('{topic}에 대한 짧은 농담을 들려줘 ')
# chain = prompt | llm | StrOutputParser()

# print(chain.invoke({'topic' : '우주여행'}))

옛날 옛적, 아주 먼 은하계에 우스꽝스러운 외계인들이 살고 있었어요. 그들은 '웃긴 우주인'이라고 불렸는데, 항상 웃음을 찾고 장난을 치길 좋아했죠. 어느 날 그들 중 한 명이 최고의 농담을 들려주기로 결심했어요! 그는 "우주의 어둠은 얼마나 심심하지?"라고 물었어요. 다른 웃긴 우주인들은 크게 웃으며 이 재미있는 이야기를 공유하고 싶어졌어요.

그 후로, 그 농담은 은하계 전체에 퍼져나가고 우주선을 통해 떠돌게 되었죠. 모든 웃긴 우주인들이 서로에게 들려주며 그들의 유쾌한 유머를 계속해서 전파했답니다. 그리고 우주의 어둠이 얼마나 심심할지에 대한 질문에 아무도 확실하게 답하지 못하지만, 웃긴 우주인들은 그들이 공유하는 농담 덕분에 결코 지루해지지 않을 거예요!


In [ ]:
# ollama create eeve-q5km -f Modelfile 
# eeve-q5km : 로컬 모델 이름

### 파인튜닝 모델 로드

songeeeeey/my-finedtuned-solar-bot

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "songeeeeey/my-finedtuned-solar-bot"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto"
)

pipe_fine_tune = pipeline("text-generation", model=model, tokenizer=tokenizer)
print(pipe_fine_tune("안녕")[0]["generated_text"])

ValueError: Unrecognized model in songeeeeey/my-finedtuned-solar-bot. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_anything, depth_